In [1]:
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.models import load_model

from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


#### TownCentre data

In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

In [3]:
#import matplotlib.pyplot as plt
#%matplotlib inline
# fig, axs = plt.subplots(1, 10, figsize=(30, 15))
# for i in range(0, 10):
#     axs[i].imshow(xtr[i])

#### Notation

$x$ - image,

$\phi$ - head angle,

$u$ - hidden variable

#### Prior network

$ p(u|x) \sim \mathcal{N}(\mu_1(x, \theta), \sigma_1(x, \theta)) $

#### Encoder network

$ q(u|x,\phi) \sim \mathcal{N}(\mu_2(x, \theta), \sigma_2(x, \theta)) $

#### Sample  $u \sim \{p(u|x), q(u|x,\phi) \}$

#### Decoder network

$p(\phi|u,x) \sim \mathcal{VM}(\mu(x,u,\theta''), \kappa(x,u,\theta'')) $

In [4]:
n_u = 8

cvae_model = CVAE(n_hidden_units=n_u)

#### Training

In [8]:
import keras
from utils.custom_keras_callbacks import SideModelCheckpoint

#proper logs format - 'logs/cvae.{epoch:02d}-{val_loss:.2f}.hdf5'

# decoder_ckpt_path = 'logs/cvae.decoder.weights.hdf5'
cvae_ckpt_path = 'logs/cvae.full_model.weights.hdf5'


model_ckpt_callback = keras.callbacks.ModelCheckpoint(cvae_ckpt_path,
                                                      monitor='val_loss',
                                                      mode='min',
                                                      save_best_only=True,
                                                      save_weights_only=True,  
                                                      verbose=1)

# decoder_ckpt_callback = SideModelCheckpoint('cvae_decoder', 
#                                             model_to_save=cvae_model.decoder_model, 
#                                             save_path=decoder_ckpt_path,
#                                             save_weights_only=True)

In [ ]:
# cvae_model.decoder_model.save_weights(decoder_ckpt_path)
# cvae_model.full_model.save_weights(cvae_ckpt_path)

In [15]:
cvae_model.full_model.fit([xtr, ytr_bit], [ytr_bit], batch_size=10, epochs=50, validation_data=([xval, yval_bit], yval_bit),
                   callbacks=[model_ckpt_callback])

Train on 6882 samples, validate on 834 samples
Epoch 1/50
6882/6882 [==============================] - 80s - loss: 1.5305 - val_loss: 1.4260
Epoch 2/50
6882/6882 [==============================] - 79s - loss: 1.4921 - val_loss: 1.7035
Epoch 3/50
6882/6882 [==============================] - 79s - loss: 1.4459 - val_loss: 1.3537
Epoch 4/50
6882/6882 [==============================] - 79s - loss: 1.3479 - val_loss: 1.2510
Epoch 5/50
6882/6882 [==============================] - 79s - loss: 1.2983 - val_loss: 1.7413
Epoch 6/50
6882/6882 [==============================] - 79s - loss: 1.2632 - val_loss: 1.4897
Epoch 7/50
6882/6882 [==============================] - 79s - loss: 1.2298 - val_loss: 1.4501
Epoch 8/50
6882/6882 [==============================] - 79s - loss: 1.1901 - val_loss: 1.6399
Epoch 9/50
6882/6882 [==============================] - 79s - loss: 1.1619 - val_loss: 1.7798
Epoch 10/50
6882/6882 [==============================] - 79s - loss: 1.1001 - val_loss: 1.1024
Epoch 11/50


6882/6882 [==============================] - 90s - loss: 0.4182 - val_loss: 0.6882
Epoch 38/50
6882/6882 [==============================] - 98s - loss: 0.4064 - val_loss: 1.0043
Epoch 39/50
6882/6882 [==============================] - 84s - loss: 0.3884 - val_loss: 1.0026
Epoch 40/50
6882/6882 [==============================] - 79s - loss: 0.3552 - val_loss: 0.7497
Epoch 41/50
6882/6882 [==============================] - 81s - loss: 0.4919 - val_loss: 0.7734
Epoch 42/50
6882/6882 [==============================] - 80s - loss: 0.4113 - val_loss: 0.7843
Epoch 43/50
6882/6882 [==============================] - 79s - loss: 0.3062 - val_loss: 0.6512
Epoch 44/50
6882/6882 [==============================] - 80s - loss: 0.3078 - val_loss: 0.7284
Epoch 45/50
6882/6882 [==============================] - 80s - loss: 0.2908 - val_loss: 0.7408
Epoch 46/50
6882/6882 [==============================] - 80s - loss: 0.2597 - val_loss: 1.0577
Epoch 47/50
6882/6882 [==============================] - 80s -

#### Predictions using decoder part

$ \phi_i = \mu(x_i,u_i,\theta'') $

In [10]:
cvae_best = CVAE(n_hidden_units=n_u)
cvae_best.full_model.load_weights(cvae_ckpt_path)

In [11]:
from scipy.stats import sem

def _eval_model(cvae_model, x, ytrue_deg, ytrue_bit, data_part):
    
    n_samples = x.shape[0]

    cvae_preds = cvae_model.full_model.predict([x, ytrue_bit])
    elbo_te, ll_te, kl_te = cvae_model._cvae_elbo_loss_np(ytrue_bit, cvae_preds)

    ypreds = cvae_model.decoder_model.predict(x)
    ypreds_bit = ypreds[:,0:2]
    kappa_preds_te = ypreds[:,2:]

    ypreds_deg = bit2deg(ypreds_bit)

    loss_te = maad_from_deg(ytrue_deg, ypreds_deg)
    mean_loss_te = np.mean(loss_te)
    std_loss_te = np.std(loss_te)

    print("MAAD error (test) : %f ± %f" % (mean_loss_te, std_loss_te))

    print("kappa (test) : %f ± %f" % (np.mean(kappa_preds_te), np.std(kappa_preds_te)))

    log_likelihood_loss = von_mises_log_likelihood_np(ytrue_bit, ypreds_bit, kappa_preds_te,
                                                         input_type='biternion')

    print("ELBO (%s) : %f ± %f SEM" % (data_part, np.mean(-elbo_te), sem(-elbo_te)))

    print("KL(encoder|prior) (%s) : %f ± %f SEM" % (data_part, np.mean(-kl_te), sem(-kl_te)))

    print("log-likelihood (%s) : %f±%fSEM" % (data_part, 
                                              np.mean(log_likelihood_loss), 
                                              sem(log_likelihood_loss)))
    return

In [12]:
_eval_model(cvae_best, xtr, ytr_deg, ytr_bit, 'train')

MAAD error (test) : 54.725613 ± 48.245662
kappa (test) : 0.934393 ± 0.810261
ELBO (train) : -1.568528 ± 0.006742 SEM
KL(encoder|prior) (train) : -0.035083 ± 0.000357 SEM
log-likelihood (train) : -1.543154±0.006767SEM


In [13]:
_eval_model(cvae_best, xval, yval_deg, yval_bit, 'validation')

MAAD error (test) : 53.554670 ± 47.014745
kappa (test) : 0.943233 ± 0.826707
ELBO (validation) : -1.570017 ± 0.018833 SEM
KL(encoder|prior) (validation) : -0.037350 ± 0.001082 SEM
log-likelihood (validation) : -1.539291±0.019358SEM


In [14]:
_eval_model(cvae_best, xte, yte_deg, yte_bit, 'test')

MAAD error (test) : 47.863967 ± 45.116663
kappa (test) : 0.903833 ± 0.801953
ELBO (test) : -1.519896 ± 0.017887 SEM
KL(encoder|prior) (test) : -0.034243 ± 0.000985 SEM
log-likelihood (test) : -1.485811±0.017619SEM
